In [1]:
import pandas as pd
import gmaps

# NEED TO SET UP API KEYS
# Let 'gkey' be the Google maps api key
# Let 'ckey' be the census api key

from config import gkey

In [2]:
# First, we need to clean the horribly formatted initial dataset into something usable.

path = 'resources/centers-uncleaned.csv'
with open(path, 'r') as csv_file:
    output = csv_file.read()
output = output.split('\n')

all_name = []
all_city = []
all_state = []
all_zip = []

for i in range(len(output)):
    info = output[i].split(';')
    try:
        name = info[1]
        # This loop searches for any entry that looks like a city / state / zip combination; if one is found it's saved, otherwise we ignore that row, since we don't have any useful information about that center.
        for j in range(2, len(info) + 1):
            try:
                if(all(x in '1234567890' for x in info[j][-5:])):
                    area = info[j]
                    break
            except:
                pass
        
        city = area.split(',')[0]
        state = area.split(',')[1][1:].split()[0]
        zipcode = area.split(',')[1][1:].split()[1]
        all_name.append(name)
        all_city.append(city)
        all_state.append(state)
        all_zip.append(zipcode)
    except:
        pass

In [3]:
# Create a nicely formatted dataframe to work with

centers_df = pd.DataFrame({'Name' : all_name, 'City' : all_city, 'State' : all_state, 'Zip' : all_zip})

zips_df = pd.read_csv('resources/zips.csv', dtype = {'ZIP' : 'str'})
zips_df = zips_df.rename(columns = {'ZIP' : 'Zip', 'LAT' : 'Lat', 'LNG' : 'Lon'})
centers_df = centers_df.merge(zips_df, on = 'Zip', how = 'inner')

out_path = 'resources/centers.csv'
centers_df.to_csv(out_path)

In [4]:
centers_df.head()

,Name,City,State,Zip,Lat,Lon
0,Henry County Day Treatment,Abbeville,AL,36310,31.595148,-85.208852
1,SpectraCare,Abbeville,AL,36310,31.595148,-85.208852
2,Rehab Day Montezuma Complex,Andalusia,AL,36420,31.172401,-86.565077
3,South Central Alabama CMHC,Andalusia,AL,36420,31.172401,-86.565077
4,Grandview Behavioral Health Center,Anniston,AL,36201,33.649160,-85.879642


In [5]:
gmaps.configure(api_key = gkey)

In [13]:
locations = centers_df[['Lat', 'Lon']].drop_duplicates()
fig = gmaps.figure()
num_centers = centers_df.groupby(by = 'Zip').count()['Name']
heat_layer = gmaps.heatmap_layer(locations, weights = num_centers, max_intensity = 100)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))